In [1]:
# 读取训练数据, 训练集包含500个模型结构，以及这些结构在cplfw，market1501，dukemtmc等8个任务上的性能排序
import json
with open('./data/CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)
print(train_data['arch1'])
print('train_num:',len(train_data.keys()))

{'cplfw_rank': 9, 'market1501_rank': 361, 'dukemtmc_rank': 426, 'msmt17_rank': 433, 'veri_rank': 327, 'vehicleid_rank': 480, 'veriwild_rank': 425, 'sop_rank': 367, 'arch': 'l231131331121121331111211121331321321'}
train_num: 500


In [2]:
import pandas as pd
import numpy as np

In [3]:
target_cols = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']

In [4]:
def get_df(train_data):
    ret = []
    for k, v in train_data.items():
        tmp = list(v['arch'])
        tmp1 = []
        for c in target_cols:
            tmp1.append(v[c])
        ret.append(tmp+tmp1+[k,v['arch']])
    retf = pd.DataFrame(ret,columns=[f'col{_}' for _ in range(len(tmp))]+target_cols+['id','arch'])
    retf['col0'] = retf['col0'].map({'l':1, 'j':2, 'k':3})
    int_cols = [x for x in retf.columns if x not in ['id','arch']]
    retf[int_cols] = retf[int_cols].astype(float)
    return retf

In [5]:
train_df = get_df(train_data)

In [6]:
from scipy.special import erfinv 

for c in target_cols:
    train_df[c+'_log1p'] = train_df[c].map(lambda x: np.log1p(x))
    train_y=train_df[c]
    mmin=np.min(train_y)+1
    mmax=np.max(train_y)+1
    train_y=np.sqrt(2) * erfinv(2 * (train_y+mmin)/(mmin+mmax)-1)
    train_df[c+'_trans_y'] = train_y

In [7]:
train_df.columns

Index(['col0', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8',
       'col9', 'col10', 'col11', 'col12', 'col13', 'col14', 'col15', 'col16',
       'col17', 'col18', 'col19', 'col20', 'col21', 'col22', 'col23', 'col24',
       'col25', 'col26', 'col27', 'col28', 'col29', 'col30', 'col31', 'col32',
       'col33', 'col34', 'col35', 'col36', 'cplfw_rank', 'market1501_rank',
       'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank',
       'veriwild_rank', 'sop_rank', 'id', 'arch', 'cplfw_rank_log1p',
       'cplfw_rank_trans_y', 'market1501_rank_log1p',
       'market1501_rank_trans_y', 'dukemtmc_rank_log1p',
       'dukemtmc_rank_trans_y', 'msmt17_rank_log1p', 'msmt17_rank_trans_y',
       'veri_rank_log1p', 'veri_rank_trans_y', 'vehicleid_rank_log1p',
       'vehicleid_rank_trans_y', 'veriwild_rank_log1p',
       'veriwild_rank_trans_y', 'sop_rank_log1p', 'sop_rank_trans_y'],
      dtype='object')

In [8]:
train_df.iloc[:3]

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,msmt17_rank_log1p,msmt17_rank_trans_y,veri_rank_log1p,veri_rank_trans_y,vehicleid_rank_log1p,vehicleid_rank_trans_y,veriwild_rank_log1p,veriwild_rank_trans_y,sop_rank_log1p,sop_rank_trans_y
0,1.0,2.0,3.0,1.0,1.0,3.0,1.0,3.0,3.0,1.0,...,6.073045,1.108919,5.793014,0.398015,6.175867,1.751613,6.054439,1.037718,5.908083,0.626575
1,3.0,1.0,1.0,1.0,2.0,2.0,1.0,3.0,2.0,1.0,...,6.192362,1.978217,5.966147,0.766946,6.030685,0.955507,6.011267,0.894120,5.713733,0.265766
2,2.0,2.0,2.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,...,5.411646,-0.132977,4.990433,-0.543441,5.198497,-0.355046,4.615121,-0.835931,3.555348,-1.476832


In [9]:
def get_step(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':1, 'j':2, 'k':3}
    time_step=12
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append(ret[:,0][:time_step])
        res1.append(ret[:,1][:time_step])
        res2.append(ret[:,2][:time_step])
        res3.append(np.array([map_[item[0]]]*time_step))
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)))
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

In [10]:
#     retf['col0'] = retf['col0'].map({'j':10, 'k':11, 'l':12})
#     retf[num_heads] = retf[num_heads].replace({1:12, 2:11, 3:10})
#     retf[mlp_ratio] = retf[mlp_ratio].replace({1:4, 2:3.5, 3:3})
#     retf[emb_dim] = retf[emb_dim].replace({1:768})

In [11]:
def get_step(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':1, 'j':2, 'k':3}
    time_step=12
    head_ = set([1+_*3 for _ in range(12)])
    mlp_ = set([2+_*3 for _ in range(12)])
    emb_ = set([3+_*3 for _ in range(12)])
    depth_ = set([0])
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append([1 if x in head_ else 0 for x in range(37)])
        res1.append([1 if x in mlp_ else 0 for x in range(37)])
        res2.append([1 if x in emb_ else 0 for x in range(37)])
        res3.append([1 if x in depth_ else 0 for x in range(37)])
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)))
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

In [12]:
get_step(train_df)

In [13]:
def get_step_0503(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':1, 'j':2, 'k':3}
    time_step=12
    head_ = set([1+_*3 for _ in range(12)])
    mlp_ = set([2+_*3 for _ in range(12)])
    emb_ = set([3+_*3 for _ in range(12)])
    depth_ = set([0])
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append([int(item[x]) if x in head_ else 0 for x in range(37)])
        res1.append([int(item[x]) if x in mlp_ else 0 for x in range(37)])
        res2.append([int(item[x]) if x in emb_ else 0 for x in range(37)])
        res3.append([map_[item[0]]]+[0]*36)
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)))
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

In [14]:
# get_step_0503(train_df)

In [15]:
len(train_df.all_emb.iloc[0])

37

In [16]:
# for c in ['head','mlp']:
#     train_df[c+'_diff1'] = train_df[c].map(lambda x: x - np.concatenate([np.array([0]),x[:-1]]))
#     train_df[c+'_diff2'] = train_df[c].map(lambda x: x - np.concatenate([np.array([0,0]),x[:-2]]))

In [17]:
from sklearn.model_selection import KFold
from tqdm import tqdm_notebook as tqdm

In [18]:
#定义数据
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self,df,use_cols,target_cols,show=0):
        self.df = df
        self.show = show
        self.use_cols = use_cols
        self.target_cols = target_cols

        self.prepare_data()
        
    def __len__(self):
        return self.df.shape[0]
    
    def prepare_data(self):
        self.y = self.df[self.target_cols].values
#         self.y = self.df[[x+'_trans_y' for x in self.target_cols]].values
        self.y0 = self.df[[self.target_cols[0]]].values
        self.y1 = self.df[[self.target_cols[1]]].values
        self.y2 = self.df[[self.target_cols[2]]].values
        self.y3 = self.df[[self.target_cols[3]]].values
        self.y4 = self.df[[self.target_cols[4]]].values
        self.y5 = self.df[[self.target_cols[5]]].values
        self.y6 = self.df[[self.target_cols[6]]].values
        self.y7 = self.df[[self.target_cols[7]]].values
        
#         self.inputs = self.df[use_cols].values
        uc = ['all_emb']
#         uc = ['head','mlp','emb']
        tmp_dt = {}
        for c in uc:
            tmp_dt[c] = np.array(self.df[c].tolist())
        self.inputs = np.concatenate([tmp_dt[c][:, None] for c in uc], 1).transpose(0, 2, 1)
        #倒序
#         b1 = a.copy()
#         b1[:,:] = b1[:,::-1]
        #横向重复
#         c = a.copy()
#         c = c.reshape(-1,1,37).repeat(37,1)
#         self.inputs = np.concatenate([a, c], -1)

        uc= ['all_emb','depth','head','mlp','emb']
        tmp_dt = {}
        for c in uc:
            tmp_dt[c] = np.array(self.df[c].tolist())
        self.inputs1= np.concatenate([tmp_dt[c][:, None] for c in uc], 1).transpose(0, 2, 1)
        
        
        if self.show==1:
            print('inputs_shape',self.inputs.shape)
            print('inputs1_shape',self.inputs1.shape)
            print('y_shape',self.y.shape)
        
    def __getitem__(self, idx):
        data = {
            "input": torch.tensor(self.inputs[idx], dtype=torch.float),
            "input1": torch.tensor(self.inputs1[idx], dtype=torch.float),
            "y": torch.tensor(self.y[idx], dtype=torch.float),
#             "y_trans": torch.tensor(self.y[idx], dtype=torch.float),
            "y0": torch.tensor(self.y0[idx], dtype=torch.float),
            "y1": torch.tensor(self.y1[idx], dtype=torch.float),
            "y2": torch.tensor(self.y2[idx], dtype=torch.float),
            "y3": torch.tensor(self.y3[idx], dtype=torch.float),
            "y4": torch.tensor(self.y4[idx], dtype=torch.float),
            "y5": torch.tensor(self.y5[idx], dtype=torch.float),
            "y6": torch.tensor(self.y6[idx], dtype=torch.float),
            "y7": torch.tensor(self.y7[idx], dtype=torch.float),
        }
        
        return data

In [19]:
train_df.columns

Index(['col0', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8',
       'col9', 'col10', 'col11', 'col12', 'col13', 'col14', 'col15', 'col16',
       'col17', 'col18', 'col19', 'col20', 'col21', 'col22', 'col23', 'col24',
       'col25', 'col26', 'col27', 'col28', 'col29', 'col30', 'col31', 'col32',
       'col33', 'col34', 'col35', 'col36', 'cplfw_rank', 'market1501_rank',
       'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank',
       'veriwild_rank', 'sop_rank', 'id', 'arch', 'cplfw_rank_log1p',
       'cplfw_rank_trans_y', 'market1501_rank_log1p',
       'market1501_rank_trans_y', 'dukemtmc_rank_log1p',
       'dukemtmc_rank_trans_y', 'msmt17_rank_log1p', 'msmt17_rank_trans_y',
       'veri_rank_log1p', 'veri_rank_trans_y', 'vehicleid_rank_log1p',
       'vehicleid_rank_trans_y', 'veriwild_rank_log1p',
       'veriwild_rank_trans_y', 'sop_rank_log1p', 'sop_rank_trans_y', 'head',
       'mlp', 'emb', 'depth', 'all_emb'],
      dtype='object')

In [20]:
train0 = train_df.sample(400, random_state=666).reset_index(drop=True)
val0 = train_df[~train_df.id.isin(train0.id)].reset_index(drop=True)

In [21]:
use_cols = [x for x in train_df.columns if 'col' in x]

In [22]:
len(use_cols)

37

In [23]:
train_dataset = MyDataset(train0, use_cols, target_cols, 1)
val_dataset = MyDataset(val0, use_cols, target_cols, 1)

inputs_shape (400, 37, 1)
inputs1_shape (400, 37, 5)
y_shape (400, 8)
inputs_shape (100, 37, 1)
inputs1_shape (100, 37, 5)
y_shape (100, 8)


In [24]:
train_dataset[0]['input']

tensor([[2.],
        [3.],
        [1.],
        [1.],
        [3.],
        [2.],
        [1.],
        [3.],
        [3.],
        [1.],
        [3.],
        [1.],
        [1.],
        [1.],
        [3.],
        [1.],
        [3.],
        [1.],
        [1.],
        [2.],
        [2.],
        [1.],
        [1.],
        [2.],
        [1.],
        [2.],
        [1.],
        [1.],
        [2.],
        [2.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [25]:
import gc
import os
import time
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup


def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    
def count_parameters(model, all=False):
    if all:
        return sum(p.numel() for p in model.parameters())
    else:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)


def save_model_weights(model, modelpath, filename):
    torch.save(model.state_dict(), modelpath+filename)
    return f"\n -> Save weights to {modelpath+filename}\n"

In [26]:
import scipy

In [27]:
def compute_metric(pred, y):
    corr = []
    if pred.shape[1]>2:
        for i in range(8):
            corr.append(scipy.stats.stats.kendalltau(pred[:, i], y[:, i])[0])
    else:
        corr.append(scipy.stats.stats.kendalltau(pred, y)[0])
    return np.array(corr)

class CVPRLoss(nn.Module):
    def __call__(self, pred, y):
        pred_mean, y_mean = pred.mean(0), y.mean(0)
        pred_std, y_std = pred.std(0), y.std(0)
        corr = ((pred - pred_mean) * (y - y_mean)).sum(0) / ((((pred - pred_mean)**2).sum(0).sqrt()) * (((y - y_mean)**2).sum(0).sqrt())) 
    
#         cos = nn.CosineSimilarity(dim=0, eps=1e-08)
#         corr1 = cos(pred, y)
#         corr2 = (corr+corr1)/2
        return 1-corr

from fast_soft_sort.pytorch_ops import soft_rank
class CVPRLoss1(nn.Module):
    #spearman
    def __call__(self, pred, y):
        pred = pred.cpu()
        y = y.cpu()
        return 1-torch.cat(
            [self.spearman(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,1) for i in range(y.shape[1])]
        ).reshape(1, -1)
        
    def corrcoef(self, target, pred):
        # np.corrcoef in torch from @mdo
        # https://forum.numer.ai/t/custom-loss-functions-for-xgboost-using-pytorch/960
        pred_n = pred - pred.mean()
        target_n = target - target.mean()
        pred_n = pred_n / pred_n.norm()
        target_n = target_n / target_n.norm()
        return (pred_n * target_n).sum()


    def spearman(self,
        target,
        pred,
        regularization="l2",
        regularization_strength=0.01,
    ):
        # fast_soft_sort uses 1-based indexing, divide by len to compute percentage of rank
        pred = soft_rank(
            pred,
            regularization=regularization,
            regularization_strength=regularization_strength,
        )
    #     print(pred)
        return self.corrcoef(target, pred / pred.shape[-1])
     
class CVPRLoss2(nn.Module):
    #kendall soft rank+tanh
    def __call__(self, pred, y):
        pred = pred.cpu()
        y = y.cpu()
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
        
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]

        tmp = soft_rank(
            preds.index_select(1, sa),
            regularization='l2',
            regularization_strength=0.01,
        )[0]-1
        
        tmp = tmp.cuda()

        score = torch.cat([((tmp[i:]-tmp[i-1]) * 10).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_softrank(nn.Module):
    #kendall soft rank+tanh
    def __call__(self, pred, y):
        pred = pred.cpu()
        y = y.cpu()
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
        
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]

        tmp = soft_rank(
            preds.index_select(1, sa),
            regularization='l2',
            regularization_strength=0.01,
        )[0]-1
        
        tmp = tmp.cuda()

        score = torch.cat([((tmp[i:]-tmp[i-1]) * 10).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_softsign(nn.Module):
    #kendall softsign
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)

    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([(tmp[i:]-tmp[i-1])/(0.01+torch.abs(tmp[i:]-tmp[i-1])) for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_tanh(nn.Module):
    # kendall tanh
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(y[:,i].reshape(1,-1), 
                           pred[:,i].reshape(1,-1)).reshape(1,-1) for i in range(y.shape[1])]
        ).reshape(1, -1)
    
    def get_score(self, actuals, preds):
        sa = actuals.argsort()[0]
        tmp = preds.index_select(1, sa.int())[0]
        score = torch.cat([((tmp[i:]-tmp[i-1])).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
    
class CVPRLoss_pair(nn.Module):
    def __call__(self, pred, y):
        return torch.cat(
            [self.pair_loss(pred[:,i], y[:,i]).reshape(1) for i in range(y.shape[1])]
        ).reshape(1,-1)
    
    def pair_loss(self, outputs, labels):
        output = outputs.unsqueeze(1)
        output1 = output.repeat(1,outputs.shape[0])
        label = labels.unsqueeze(1)
        label1 = label.repeat(1,labels.shape[0])

        tmp = (output1-output1.t())*torch.sign(label1-label1.t())
        tmp = torch.log(1+torch.exp(-tmp))
        eye_tmp = tmp*torch.eye(len(tmp)).cuda()
        new_tmp = tmp - eye_tmp
        loss = torch.sum(new_tmp)/(outputs.shape[0]*(outputs.shape[0]-1))

        return loss
    
class CVPRLoss_tanh1(nn.Module):
    # kendall tanh
    def __call__(self, pred, y):
        return 1-torch.cat(
            [self.get_score(pred[:,i], y[:,i]).reshape(1) for i in range(y.shape[1])]
        ).reshape(1,-1)
    
    def get_score(self, outputs, labels):
        output1 = outputs.unsqueeze(1).repeat(1,outputs.shape[0])
        label1 = labels.unsqueeze(1).repeat(1,labels.shape[0])

        tmp = ((output1-output1.t())*torch.sign(label1-label1.t())).tanh()
        eye_tmp = tmp*torch.eye(tmp.shape[0]).cuda()
        new_tmp = tmp - eye_tmp
        
        loss = torch.sum(new_tmp)/(outputs.shape[0]*(outputs.shape[0]-1))

        return loss

In [28]:
def train(model, 
        train_dataset, 
        val_dataset, 
        verbose=20, 
        fold_=0,
        modelname='MLP_base',
        modelpath=r'./model'+'//',
        input='input',
        y='y',
        early_stop_round=60,
        debug=False):
    
    print(f'Model parameters count: {count_parameters(model)}')
    #数据加载
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=True,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )
    print(f'train batch num: {len(train_loader)}')
    print(f'val batch num: {len(val_loader)}')
            
    # Optimizer
    optimizer = getattr(torch.optim, optim)(model.parameters(), lr=LR)
    # Scheduler
    num_warmup_steps = int(0.1 * EPOCHS * len(train_loader))
    num_training_steps = int(EPOCHS * len(train_loader))
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps, num_training_steps
    )
    print(f'optim: {optim}, lr: {LR}, warmup_steps: {num_warmup_steps}')
    
    
    #train
    bst_epoch=0
    score_best=0
    first_epoch_eval=0
    for epoch in range(EPOCHS):
        if epoch > early_stop_round and (epoch - bst_epoch > early_stop_round):
            print(f'early stopping.')
            break
            
        model.train()
        model.zero_grad()
        start_time = time.time()

        avg_loss = 0
        for data in train_loader:
            pred = model(data[input].to(device))

            loss = loss_fct(
                pred,
                data[y].to(device)
            ).mean()

            loss.backward()
            avg_loss += loss.item() / len(train_loader)

            optimizer.step()
            scheduler.step()

            optimizer.zero_grad(set_to_none=True)

        #VAL
        model.eval()
        mae, avg_val_loss = 0, 0
        preds = []

        with torch.no_grad():
            for data in val_loader:
                pred = model(data[input].to(device))

                loss = loss_fct(
                    pred,
                    data[y].to(device)
                ).mean()

                avg_val_loss += loss.item() / len(val_loader)

                preds.append(pred.detach().cpu().numpy())

        preds = np.concatenate(preds, 0)
        if y=='y':
            mae = compute_metric(preds,val_dataset.df[target_cols].values).mean()
        else:
            mae = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
        
        elapsed_time = time.time() - start_time
        if (epoch + 1) % verbose == 0:
            elapsed_time = elapsed_time * verbose
            lr = scheduler.get_last_lr()[0]
    #         lr=LR
            print(
                f"Epoch {epoch + 1:02d}/{ EPOCHS:02d} \t lr={lr:.1e}\t t={elapsed_time:.0f}s \t"
                f"loss={avg_loss:.4f}",
                end="\t",
            )

            if (epoch + 1 >= first_epoch_eval) or (epoch + 1 == EPOCHS):
                print(f"val_loss={avg_val_loss:.4f}\tcorr={mae:.4f}")
            else:
                print("")
                
        #保存最优模型
        if mae>score_best:
            bst = save_model_weights(model, modelpath, f'{modelname}_{fold_}.pt')
            score_best = mae
            bst_epoch = epoch
            if y=='y':
                bst_list = compute_metric(preds,val_dataset.df[target_cols].values)
            else:
                bst_list = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
            bst_preds = preds
    print(f'best score {score_best}, best epoch: {bst_epoch}, {bst} ' )
    print(bst_list,'\n\n')
    del (val_loader, train_loader, loss, data, pred)
    gc.collect()
    torch.cuda.empty_cache()
    
    return bst_preds, bst_list

In [38]:
class CVPRModel(nn.Module):
    def __init__(
        self,
        input_dim=3,
        num_classes=8,
        time_step=12,
        bi=True
    ):
        super(CVPRModel,self).__init__()
        
        self.bi_num = 2 if bi else 1
        self.time_step = time_step
        
        self.MLP = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
        )
        
#         self.MLP = nn.Sequential(
#             nn.Linear(time_step, 128),
#             nn.ReLU(),
#             nn.Linear(128, 256),
#             nn.ReLU(),
#             nn.Linear(256, 256)
#         )

        
        self.LSTM0 = nn.LSTM(256, 256, 
                    batch_first=True, 
                    bidirectional=bi, 
                    num_layers=1,
                    dropout=0)
                                 
        
        self.LSTM1 = nn.LSTM(512, 256, 
                    batch_first=True, 
                    bidirectional=bi, 
                    num_layers=1,
                    dropout=0
                            )    

#         self.LSTM2 = nn.LSTM(512, 256, 
#                     batch_first=True, 
#                     bidirectional=bi, 
#                     num_layers=1,
#                     dropout=0)
        
        self.Logits = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear((512+512+256+1)*self.time_step, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        x1 = self.MLP(x)
        x2, (h0, c0) = self.LSTM0(x1)
        x3, (h1, c1) = self.LSTM1(x2, (h0, c0))
#         x4, (h2, c2) = self.LSTM2(x3, (h1, c1))
        c1 = torch.cat([x3, x2, x1, x], -1)
        pred = self.Logits(c1)
        return pred

In [31]:
#参数设置
seed = 666
BATCH_SIZE = 400
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.001
optim = "Adam"
device = "cuda" if torch.cuda.is_available() else "cpu"
modelpath = r'./model'+'//'

In [32]:
a = torch.rand(128,37,1).to(device)
net = CVPRModel(input_dim=1,
                num_classes=len(target_cols),
                bi=True,
                time_step=37
               ).to(device)
b = net(a)
b.shape

torch.Size([128, 8])

In [33]:
b[0].shape

torch.Size([8])

In [34]:
b.flatten(1).shape

torch.Size([128, 8])

In [35]:
del a,b,net;
gc.collect()
torch.cuda.empty_cache()

In [74]:
#参数设置
seed = 666
BATCH_SIZE = 400
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.001
optim = "Adam"
device = "cuda" if torch.cuda.is_available() else "cpu"
modelpath = r'E:\cvpr_model'+'//'

# loss_fct = CVPRLoss_tanh()
loss_fct = CVPRLoss_tanh1()
index = '_t'
seed_everything(seed)
modelname = 'LSTM1_step37_base1_test'
model = CVPRModel(input_dim=1,
                num_classes=8,
                bi=True,
                time_step=37
               ).to(device)
preds,_ = train(model, 
            train_dataset, 
            val_dataset, 
            verbose=20, 
            fold_=index,
            modelname=modelname,
            modelpath=modelpath,
            input='input',
            y='y',
            debug=False
             )

Model parameters count: 102162440
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.001, warmup_steps: 30
Epoch 20/300 	 lr=6.7e-04	 t=1s 	loss=0.4693	val_loss=0.4886	corr=0.5151
Epoch 40/300 	 lr=9.6e-04	 t=2s 	loss=0.3250	val_loss=0.3550	corr=0.6465
Epoch 60/300 	 lr=8.9e-04	 t=2s 	loss=0.2366	val_loss=0.2599	corr=0.7410
Epoch 80/300 	 lr=8.1e-04	 t=2s 	loss=0.1868	val_loss=0.2126	corr=0.7890
Epoch 100/300 	 lr=7.4e-04	 t=1s 	loss=0.1779	val_loss=0.2099	corr=0.7911
Epoch 120/300 	 lr=6.7e-04	 t=1s 	loss=0.1690	val_loss=0.2140	corr=0.7856
Epoch 140/300 	 lr=5.9e-04	 t=1s 	loss=0.1570	val_loss=0.2164	corr=0.7841
early stopping.
best score 0.7926767676767678, best epoch: 82, 
 -> Save weights to E:\cvpr_model//LSTM1_step37_base1_test__t.pt
 
[0.34545455 0.87676768 0.89010101 0.96161616 0.87717172 0.67434343
 0.91272727 0.80323232] 




In [75]:
loss_fct = CVPRLoss_tanh()
index = '_t'
seed_everything(seed)
modelname = 'LSTM1_step37_base1_test'
model = CVPRModel(input_dim=1,
                num_classes=8,
                bi=True,
                time_step=37
               ).to(device)
preds,_ = train(model, 
            train_dataset, 
            val_dataset, 
            verbose=20, 
            fold_=index,
            modelname=modelname,
            modelpath=modelpath,
            input='input',
            y='y',
            debug=False
             )

Model parameters count: 102162440
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.001, warmup_steps: 30
Epoch 20/300 	 lr=6.7e-04	 t=8s 	loss=0.4608	val_loss=0.4726	corr=0.5297
Epoch 40/300 	 lr=9.6e-04	 t=9s 	loss=0.3205	val_loss=0.3562	corr=0.6443
Epoch 60/300 	 lr=8.9e-04	 t=8s 	loss=0.2324	val_loss=0.2522	corr=0.7494
Epoch 80/300 	 lr=8.1e-04	 t=8s 	loss=0.1889	val_loss=0.2102	corr=0.7910
Epoch 100/300 	 lr=7.4e-04	 t=8s 	loss=0.1768	val_loss=0.2083	corr=0.7928
Epoch 120/300 	 lr=6.7e-04	 t=9s 	loss=0.1649	val_loss=0.2127	corr=0.7886
Epoch 140/300 	 lr=5.9e-04	 t=8s 	loss=0.1450	val_loss=0.2270	corr=0.7732
early stopping.
best score 0.7956060606060608, best epoch: 93, 
 -> Save weights to E:\cvpr_model//LSTM1_step37_base1_test__t.pt
 
[0.34909091 0.87676768 0.89171717 0.95757576 0.88767677 0.67676768
 0.91393939 0.81131313] 




In [40]:
# [0.25979798 0.84808081 0.91191919 0.9620202  0.90585859 0.68767677 0.93171717 0.78585859] 

In [41]:
# 0.7689898989898991 [0.27191919 0.85818182 0.88080808 0.93616162 0.85575758 0.6759596 0.86949495 0.80363636] 

In [42]:
# 2层自带lstm 0.7911616161616162 
# 1层的输出作为2层的h和c 0.7920707070707071
# 2层手写，不初始化 0.7911616161616162 
# 3层手写，输出作为输入 0.7911111111111113 
# 2自带*2自带，输出作为输入 0.7875757575757577 
# 1层输出作为2层输入，最后2层输出cat 0.7894444444444446
# 1层输出作为2层输入，最后2层输出相乘 0.7885858585858587
# 1层的输出作为2层的h和c,加一层gru cat 0.7894949494949497 
# 1层的输出作为2层的h和c,加一层gru*1层输出 cat 0.7886868686868689
# 1层的输出作为2层的h和c,加2层gru cat 0.7913131313131314 
# 1层的输出作为2层的h和c,加2层gru*1层输出 cat2层输出 0.7915151515151516 
# 1层的输出作为2层的h和c，2层的输出transformer然后乘以2层的输出，0.7899494949494951
# 1层的输出作为2层的h和c,Tanh 0.7893939393939395 
# 1层的输出作为2层的h和c,relu 0.7862121212121214 
# 1层的输出作为2层的h和c,fc出去，0.7872222222222224 

In [43]:
# index = '_t'
# modelname = 'LSTM1_step37_base1_test'
# scoref = []
# for i in range(8):
#     seed_everything(seed)
#     model = CVPRModel(input_dim=1,
#                     num_classes=1,
#                     bi=True,
#                     time_step=37
#                    ).to(device)
#     preds,_ = train(model, 
#                 train_dataset, 
#                 val_dataset, 
#                 verbose=20, 
#                 fold_=index,
#                 modelname=modelname,
#                 modelpath=modelpath,
#                 input='input',
#                 y=f'y{i}',
#                 debug=False
#                  )
#     scoref.append(_)
# np.mean(scoref)

In [44]:
# [0.33171717 0.86222222 0.88888889 0.94222222 0.8630303  0.66909091 0.9030303  0.79757576]

In [55]:
# a = torch.arange(-50,50,0.1)
# b = a.tanh()
# df = pd.DataFrame()
# df['a'] = a.detach().numpy()
# df['b'] = a.tanh()
# df['b0'] = (a*2).tanh()
# df['b1'] = (a*0.1).tanh()
# df.plot(x='a',y=['b','b0','b1'])

In [39]:
def train_sig(model, 
        train_dataset, 
        val_dataset, 
        verbose=20, 
        fold_=0,
        modelname='MLP_base',
        modelpath=r'./model'+'//',
        input='input',
        y='y',
        early_stop_round=60,
        debug=False):
    
    print(f'Model parameters count: {count_parameters(model)}')
    #数据加载
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=True,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )
    print(f'train batch num: {len(train_loader)}')
    print(f'val batch num: {len(val_loader)}')
            
    # Optimizer
    optimizer = getattr(torch.optim, optim)(model.parameters(), lr=LR)
    # Scheduler
    num_warmup_steps = int(0.1 * EPOCHS * len(train_loader))
    num_training_steps = int(EPOCHS * len(train_loader))
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps, num_training_steps
    )
    print(f'optim: {optim}, lr: {LR}, warmup_steps: {num_warmup_steps}')
    
    
    #train
    bst_epoch={_:0 for _ in range(8)}
    score_best={_:0 for _ in range(8)}
    first_epoch_eval=0
    for epoch in range(EPOCHS):
        
        model.train()
        model.zero_grad()
        start_time = time.time()

        avg_loss = 0
        for data in train_loader:
            pred = model(data[input].to(device))
#             print(pred.shape,data['y'].shape)

            loss = loss_fct(
                pred,
                data[y].to(device)
            ).mean()

            loss.backward()
            avg_loss += loss.item() / len(train_loader)

            optimizer.step()
            scheduler.step()

            optimizer.zero_grad(set_to_none=True)

        #VAL
        model.eval()
        mae, avg_val_loss = 0, 0
        preds = []

        with torch.no_grad():
            for data in val_loader:
                pred = model(data[input].to(device))

                loss = loss_fct(
                    pred,
                    data[y].to(device)
                ).mean()

                avg_val_loss += loss.item() / len(val_loader)

                preds.append(pred.detach().cpu().numpy())

        preds = np.concatenate(preds, 0)
        if y=='y':
            mae = compute_metric(preds,val_dataset.df[target_cols].values).mean()
        else:
            mae = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
        
        elapsed_time = time.time() - start_time
        if (epoch + 1) % verbose == 0:
            elapsed_time = elapsed_time * verbose
            lr = scheduler.get_last_lr()[0]
    #         lr=LR
            print(
                f"Epoch {epoch + 1:02d}/{ EPOCHS:02d} \t lr={lr:.1e}\t t={elapsed_time:.0f}s \t"
                f"loss={avg_loss:.4f}",
                end="\t",
            )

            if (epoch + 1 >= first_epoch_eval) or (epoch + 1 == EPOCHS):
                print(f"val_loss={avg_val_loss:.4f}\tcorr={mae:.4f}")
            else:
                print("")
                
        #保存最优模型
        score1 = compute_metric(preds,val_dataset.df[target_cols].values)
        for i in range(8):
            if score1[i] > score_best[i]:
                bst = save_model_weights(model, modelpath, f'{modelname}_target{i}_{fold_}.pt')
                score_best[i] = score1[i]
                bst_epoch[i] = epoch
#                 print(f'target{i} best score {score_best[i]}, best epoch: {bst_epoch[i]}, {bst} ' )
                
    bst_list = [score_best[i] for i in range(8)]
    print(bst_list,'\n\n')
    del (val_loader, train_loader, loss, data, pred)
    gc.collect()
    torch.cuda.empty_cache()
    
    return bst_list


In [40]:
# K折训练，同时
seed = 666
BATCH_SIZE = 400
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.001
optim = "Adam"
device = "cuda" if torch.cuda.is_available() else "cpu"
modelpath = r'E:\cvpr_model'+'//'

loss_fct = CVPRLoss_tanh1()
# loss_fct = CVPRLoss_pair()
k=5
scoref = []
skf = KFold(n_splits=k, shuffle=False)
for index, (train_index, test_index) in enumerate(skf.split(train_df)):   
    print(f'FOLD {index}')
    train0 = train_df.iloc[train_index]
    val0 = train_df.iloc[test_index]   
    train_dataset = MyDataset(train0, use_cols, target_cols)
    val_dataset = MyDataset(val0, use_cols, target_cols)
    print(f'train size: {len(train0)}, val size: {len(val0)}')

    modelname = f'lstm2y稠密_bc400_lr001_tanh_mat'
    seed_everything(seed)
    model = CVPRModel(input_dim=1,
                    num_classes=8,
                    bi=True,
                    time_step=37
                   ).to(device)

    _ = train_sig(model, 
                train_dataset, 
                val_dataset, 
                verbose=10, 
                fold_=index,
                modelname=modelname,
                modelpath=modelpath,
                input='input',
                y='y',
                debug=False
                 )
    scoref.append(_)
scoreff = scoref
print(np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)])

FOLD 0
train size: 400, val size: 100
Model parameters count: 102162440
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.001, warmup_steps: 30
Epoch 10/300 	 lr=3.3e-04	 t=1s 	loss=0.9267	val_loss=0.9024	corr=0.3262
Epoch 20/300 	 lr=6.7e-04	 t=1s 	loss=0.5197	val_loss=0.5445	corr=0.4587
Epoch 30/300 	 lr=1.0e-03	 t=4s 	loss=0.3986	val_loss=0.4288	corr=0.5718
Epoch 40/300 	 lr=9.6e-04	 t=3s 	loss=0.3292	val_loss=0.3565	corr=0.6444
Epoch 50/300 	 lr=9.3e-04	 t=4s 	loss=0.2842	val_loss=0.3075	corr=0.6940
Epoch 60/300 	 lr=8.9e-04	 t=4s 	loss=0.2360	val_loss=0.2532	corr=0.7487
Epoch 70/300 	 lr=8.5e-04	 t=3s 	loss=0.1987	val_loss=0.2217	corr=0.7803
Epoch 80/300 	 lr=8.1e-04	 t=1s 	loss=0.1874	val_loss=0.2136	corr=0.7880
Epoch 90/300 	 lr=7.8e-04	 t=1s 	loss=0.1840	val_loss=0.2149	corr=0.7856
Epoch 100/300 	 lr=7.4e-04	 t=1s 	loss=0.1819	val_loss=0.2140	corr=0.7867
Epoch 110/300 	 lr=7.0e-04	 t=1s 	loss=0.1794	val_loss=0.2148	corr=0.7858
Epoch 120/300 	 lr=6.7e-04	 t=1s 	loss=0.1777	

In [41]:
# 0.7 0.77966 [0.26691, 0.86489, 0.8943, 0.96145, 0.89042, 0.65576, 0.912, 0.79152]
(0.7803, [0.26618, 0.86804, 0.89471, 0.95782, 0.89059, 0.66764, 0.90909, 0.78836])

(0.7803,
 [0.26618, 0.86804, 0.89471, 0.95782, 0.89059, 0.66764, 0.90909, 0.78836])

In [42]:
# loss_fct = CVPRLoss_softsign()
# k=5
# scoref = []
# skf = KFold(n_splits=k, shuffle=False)
# for index, (train_index, test_index) in enumerate(skf.split(train_df)):   
#     print(f'FOLD {index}')
#     train0 = train_df.iloc[train_index]
#     val0 = train_df.iloc[test_index]   
#     train_dataset = MyDataset(train0, use_cols, target_cols)
#     val_dataset = MyDataset(val0, use_cols, target_cols)
#     print(f'train size: {len(train0)}, val size: {len(val0)}')

#     modelname = f'LSTM_2layer_step37_kendall_8all_softsign'
#     seed_everything(seed)
#     model = CVPRModel(input_dim=1,
#                     num_classes=8,
#                     bi=True,
#                     time_step=37
#                    ).to(device)
#     preds,_ = train(model, 
#                 train_dataset, 
#                 val_dataset, 
#                 verbose=100, 
#                 fold_=index,
#                 modelname=modelname,
#                 modelpath=modelpath,
#                 input='input',
#                 y='y',
#                 debug=False
#                  )
#     scoref.append(_)
# scoreff = scoref
# np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)]

In [43]:
np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)]

(0.78961,
 [0.2876, 0.87321, 0.90311, 0.9659, 0.9002, 0.67071, 0.91895, 0.79717])

In [44]:
# (0.78795,
#  [0.26836, 0.87111, 0.90545, 0.96509, 0.89745, 0.67507, 0.91895, 0.8021])

In [45]:
#加载test
with open('./data/CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)
test_data['arch99997']

{'cplfw_rank': 0,
 'market1501_rank': 0,
 'dukemtmc_rank': 0,
 'msmt17_rank': 0,
 'veri_rank': 0,
 'vehicleid_rank': 0,
 'veriwild_rank': 0,
 'sop_rank': 0,
 'arch': 'j121221121221221311331321121221000000'}

In [46]:
test_df = get_df(test_data)
get_step(test_df)

In [47]:
def pred_cuda(test_df, model):
    #获得预测
    test_dataset = MyDataset(test_df, use_cols, target_cols, 0)
    test_loader = DataLoader(
        test_dataset,
        batch_size=1024*8,
        shuffle=False,
        drop_last=False,
        pin_memory=True
    )

    preds = []
    model.eval()
    with torch.no_grad():
        for data in tqdm(test_loader):
            pred = model(data['input'].to(device))
            preds.append(pred.detach().cpu().numpy())
    preds = np.concatenate(preds, 0)
    print(preds.shape)

    del test_dataset, test_loader, model;
    gc.collect()
    torch.cuda.empty_cache()
    return preds
        
def predict_sig(test_df, k=5, modelname=''):
    for target in range(8):        
        cols = []
        for fold_ in range(k):
            modelname1 = f'{modelname}_target{target}_{fold_}.pt'
            print(f'Model {modelname1}')
            model = CVPRModel(input_dim=1,
                        num_classes=8,
                        bi=True,
                        time_step=37
                       ).to(device)
            model.load_state_dict(torch.load(modelpath+modelname1))

            pred_ = pred_cuda(test_df, model)[:, target]
            tmp_c = f'{target_cols[target]}_{fold_}'
            test_df[tmp_c] = pred_
            cols.append(tmp_c)
            print(f'Done {tmp_c}')
            
        print(cols)
        test_df[cols] = test_df[cols].rank()
        test_df[target_cols[target]] = test_df[cols].mean(axis=1).rank()
        
def predict_all(test_df, k=5, modelname = f'lstm2y稠密_bc400_lr001_softsign'):

    print(f'Model {modelname}')

    cols = []
    for fold_ in range(k):
        model = CVPRModel(input_dim=1,
                    num_classes=8,
                    bi=True,
                    time_step=37
                   ).to(device)
        model.load_state_dict(torch.load(modelpath+f'{modelname}_{fold_}.pt'))

        pred_ = pred_cuda(test_df, model)
        tmp_c = [f'{target}_{fold_}' for target in target_cols]
        test_df[tmp_c] = pred_
        cols += tmp_c
    #get rank
    print(cols)
    test_df[cols] = test_df[cols].rank()
    for c in target_cols:
        test_df[c] = test_df[[f'{c}_{fold_}' for fold_ in range(k)]].mean(axis=1)
        test_df[c] = test_df[c].rank()

In [48]:
k, modelname

(5, 'lstm2y稠密_bc400_lr001_tanh_mat')

In [79]:
# LSTM_2layer_step37_kendall_8all_softrankloss
# LSTM_2layer_step37_kendall_8all_softsign
# LSTM_2layer_step37_kendall_8all_tanh


In [49]:
train_df1 = train_df.copy()
predict_sig(train_df1, k=5, modelname='lstm2y稠密_bc400_lr001_tanh_mat')
compute_metric(train_df[target_cols].values, train_df1[target_cols].values)

Model lstm2y稠密_bc400_lr001_tanh_mat_target0_0.pt


C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/1 [00:00<?, ?it/s]

(500, 8)
Done cplfw_rank_0
Model lstm2y稠密_bc400_lr001_tanh_mat_target0_1.pt


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

(500, 8)
Done sop_rank_3
Model lstm2y稠密_bc400_lr001_tanh_mat_target7_4.pt


  0%|          | 0/1 [00:00<?, ?it/s]

(500, 8)
Done sop_rank_4
['sop_rank_0', 'sop_rank_1', 'sop_rank_2', 'sop_rank_3', 'sop_rank_4']


array([0.37461616, 0.90014512, 0.91819605, 0.97460526, 0.92847466,
       0.72192225, 0.92732675, 0.83744911])

In [50]:
np.mean([0.37461616, 0.90014512, 0.91819605, 0.97460526, 0.92847466,
       0.72192225, 0.92732675, 0.83744911])

0.82284192

In [51]:
predict_sig(test_df, k=5, modelname='lstm2y稠密_bc400_lr001_tanh_mat')

Model lstm2y稠密_bc400_lr001_tanh_mat_target0_0.pt


C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)
Done cplfw_rank_0
Model lstm2y稠密_bc400_lr001_tanh_mat_target0_1.pt


  0%|          | 0/13 [00:00<?, ?it/s]

...

(99500, 8)
Done sop_rank_2
Model lstm2y稠密_bc400_lr001_tanh_mat_target7_3.pt


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)
Done sop_rank_3
Model lstm2y稠密_bc400_lr001_tanh_mat_target7_4.pt


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)
Done sop_rank_4
['sop_rank_0', 'sop_rank_1', 'sop_rank_2', 'sop_rank_3', 'sop_rank_4']


In [43]:
# predict_all(test_df, k=5, modelname='lstm2y稠密_bc400_lr001_pair')

Model lstm2y稠密_bc400_lr001_softsign


C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)


  0%|          | 0/13 [00:00<?, ?it/s]

...

(99500, 8)


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)


  0%|          | 0/13 [00:00<?, ?it/s]

(99500, 8)
['cplfw_rank_0', 'market1501_rank_0', 'dukemtmc_rank_0', 'msmt17_rank_0', 'veri_rank_0', 'vehicleid_rank_0', 'veriwild_rank_0', 'sop_rank_0', 'cplfw_rank_1', 'market1501_rank_1', 'dukemtmc_rank_1', 'msmt17_rank_1', 'veri_rank_1', 'vehicleid_rank_1', 'veriwild_rank_1', 'sop_rank_1', 'cplfw_rank_2', 'market1501_rank_2', 'dukemtmc_rank_2', 'msmt17_rank_2', 'veri_rank_2', 'vehicleid_rank_2', 'veriwild_rank_2', 'sop_rank_2', 'cplfw_rank_3', 'market1501_rank_3', 'dukemtmc_rank_3', 'msmt17_rank_3', 'veri_rank_3', 'vehicleid_rank_3', 'veriwild_rank_3', 'sop_rank_3', 'cplfw_rank_4', 'market1501_rank_4', 'dukemtmc_rank_4', 'msmt17_rank_4', 'veri_rank_4', 'vehicleid_rank_4', 'veriwild_rank_4', 'sop_rank_4']


In [52]:
test_df[target_cols] = test_df[target_cols].astype(int)-1

In [53]:
test_df[target_cols].describe()

,cplfw_rank,market1501_rank,dukemtmc_rank,msmt17_rank,veri_rank,vehicleid_rank,veriwild_rank,sop_rank
count,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000
mean,49749.420291,49749.431859,49749.435447,49749.426925,49749.433176,49749.427518,49749.435327,49749.427528
std,28723.320935,28723.320063,28723.320508,28723.320596,28723.320212,28723.319067,28723.320314,28723.320348
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24874.750000,24874.500000,24874.750000,24874.750000,24874.750000,24874.000000,24874.000000,24874.000000
50%,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000,49749.500000
75%,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000
max,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000


In [54]:
test_df[target_cols].nunique()

cplfw_rank         90502
market1501_rank    91932
dukemtmc_rank      92333
msmt17_rank        91219
veri_rank          92062
vehicleid_rank     91418
veriwild_rank      92327
sop_rank           91358
dtype: int64

In [55]:
modelname

'lstm2y稠密_bc400_lr001_tanh_mat'

In [56]:
modelname = 'lstm2y稠密_bc400_lr001_tanh_mat_20220514'

In [57]:
#输出
for i in tqdm(test_df[['id']+target_cols].values):
    id_ = i[0]
    for k,v in enumerate(target_cols):
        k += 1
        test_data[id_][v] = i[k]
        
with open(f'./sub/{modelname}.json', 'w') as f:
    json.dump(test_data, f)

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/99500 [00:00<?, ?it/s]

In [58]:
modelname

'lstm2y稠密_bc400_lr001_tanh_mat_20220514'